In [1]:
import os
if not os.path.exists('ShoeV2_photo'):
    !wget https://www.dropbox.com/s/g6b6gtvmdu0h77x/ShoeV2_photo.zip
    !unzip -q ShoeV2_photo.zip

--2022-11-25 14:15:46--  https://www.dropbox.com/s/g6b6gtvmdu0h77x/ShoeV2_photo.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/g6b6gtvmdu0h77x/ShoeV2_photo.zip [following]
--2022-11-25 14:15:47--  https://www.dropbox.com/s/raw/g6b6gtvmdu0h77x/ShoeV2_photo.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3a362d19195e2744b697877c23.dl.dropboxusercontent.com/cd/0/inline/BxYvBq2878vyjnuYlKl930rshSIVyblMwwrhFFXdnIMrX52cI9m403RE12gVpZl-P_5l9wz8QysRu8Y_4fFOwcyVRPZOk6ColIcAt24s_gD8V5g-z1nYZjQYbj0n5oEUEuqMH9rcX3EAeVOqaD01XX8ZK4NRdVVSFbOyQyMmd-ElKA/file# [following]
--2022-11-25 14:15:48--  https://uc3a362d19195e2744b697877c23.dl.dropboxusercontent.com/cd/0/inline/BxYvBq2878vyjnuYlKl930rshSIVyblMwwrhFFXdnIMrX52cI9m403RE12gVpZl-P

In [2]:
!pip install torch_snippets
from torch_snippets import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 58 kB 7.0 MB/s 
     |████████████████████████████████| 237 kB 68.4 MB/s 
     |████████████████████████████████| 78 kB 9.3 MB/s 
     |████████████████████████████████| 1.6 MB 69.9 MB/s 
     |████████████████████████████████| 175 kB 72.1 MB/s 
     |████████████████████████████████| 2.2 MB 71.4 MB/s 
     |████████████████████████████████| 51 kB 8.4 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=084e6457a6d2b48f660a22edd9f6720ecfddd2aa9c36b2e7bb1a851d649beb23
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26323 sha256=9a03a0a956fee0aba9d9c38425015276993b54a5740e4750d30f3ef68e840d34
  Stored in directory: /root/.c

In [3]:
def sketch_edges(img):
  # RGB Image to Gray Image
  rgb2gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  # 양방향필터: Gaussian 필터를 양방향으로 적용해 물체선을 완만하게하고 블러링을 어느정도 조절함.
  img_gray = cv2.bilateralFilter(rgb2gray,5,50,50)
  # Edge detection 테크닉을 통해 스케치 이미지 생성
  img_edge = cv2.Canny(img_gray,45,100)
  # Invert the binary points
  img_edge = cv2.bitwise_not(img_gray)
  # Gray Image to RGB Image
  img_edges = cv2.cvtColor(img_edge,cv2.COLOR_GRAY2RGB)
  return img_edges

IMAGE_SIZE = 256

In [4]:
# width x height x channel 재배치
preprocessing = T.Compose([
    T.Lambda(lambda x: torch.Tensor(x.copy()).permute(2,0,1).to(device))
])
# 정규화
normalize = lambda x: (x - 127.5)/127.5

In [5]:
class dataset(Dataset):
  def __init__(self,imgs):
    self.imgs = imgs # 이미지 입력
  def __len__(self): return len(self.imgs) # 총 이미지 수 파악
  def __getitem__(self,ix):
    fetch = self.imgs[ix]
    try:
      img = read(fetch,1) 
    except:
      blank = preprocessing(Blank(IMAGE_SIZE,IMAGE_SIZE,3))
    img_edges = sketch_edges(img) # 기존 이미지 -> 엣지 이미지로 변환

    # 기존 이미지, 엣지 이미지 리사이징&정규화
    img, edges = normalize(resize(img,IMAGE_SIZE)), normalize(resize(img_edges, IMAGE_SIZE))

    # 엣지 이미지에 원모양 색깔 흩뿌리기
    self.__sprinkle_dotted_color_circles_on_edge_img(edges,img)
    img, edges = preprocessing(img), preprocessing(edges)
    return edges, img
  
  def __sprinkle_dotted_color_circles_on_edge_img(self,img_src,img_target):
    black_coords = self._get_non_white_coordinates(img_target)
    for center_y, center_x in black_coords:
      self._draw_color_circle_on_src_img(img_src,img_target,center_y, center_x)

  def _get_non_white_coordinates(self, img):
  
    non_white_mask = np.sum(img, axis=-1) < 2.75 #Channel Sum
    non_white_y, non_white_x = np.nonzero(non_white_mask) #Returns the x,y coordinates in the form of (array([x0,x1,x2,...,xn]),array([y0,y1,y2,...,yn]))
    # randomly sample non-white coordinates
    n_non_white = len(non_white_y)
    n_color_points = min(n_non_white, 300)
    idxs = np.random.choice(n_non_white, n_color_points, replace=False) #Generates "n_color_points" random indices in the range of "n_non_white"
    non_white_coords = list(zip(non_white_y[idxs], non_white_x[idxs]))
    return non_white_coords #Final random x,y coordinates

  def _draw_color_circle_on_src_img(self, img_src, img_target, center_y, center_x):
    assert img_src.shape == img_target.shape, "Image source and target must have same shape."
    y0, y1, x0, x1 = self._get_color_point_bbox_coords(center_y, center_x)
    color = np.mean(img_target[y0:y1, x0:x1], axis=(0, 1))
    img_src[y0:y1, x0:x1] = color #해당 부분 색깔 적용

  def _get_color_point_bbox_coords(self, center_y, center_x):
    radius = 2
    y0 = max(0, center_y-radius+1)
    y1 = min(IMAGE_SIZE, center_y+radius)
    x0 = max(0, center_x-radius+1)
    x1 = min(IMAGE_SIZE, center_x+radius)
    return y0, y1, x0, x1 #바운딩박스 좌표값

  def choose(self): return self[randint(len(self))]


In [6]:
from sklearn.model_selection import train_test_split
train_imgs, val_imgs = train_test_split(Glob('/content/ShoeV2_photo/*.png'), test_size=0.2, random_state=2) #트레이닝 세트/테스트 세트 배분
trn_ds, val_ds = dataset(train_imgs), dataset(val_imgs)

trn_dl = DataLoader(trn_ds,batch_size=32,shuffle=True)
val_dl = DataLoader(val_ds,batch_size=32,shuffle=True)

inspect(*next(iter(trn_dl)))

══════════════════════════════════════════════════════════════════

Tensor  Shape: torch.Size([32, 3, 256, 256])    Min: -1.000     Max: 0.999      Mean: -0.564    dtype: 
torch.float32 @ cuda:0

══════════════════════════════════════════════════════════════════

Tensor  Shape: torch.Size([32, 3, 256, 256])    Min: -1.000     Max: 1.000      Mean: 0.557     dtype: 
torch.float32 @ cuda:0

══════════════════════════════════════════════════════════════════

In [7]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

class UNetDown(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, dropout=0.0):
        super(UNetDown, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 4, 2, 1, bias=False)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_size))
        layers.append(nn.LeakyReLU(0.2))
        if dropout:
            layers.append(nn.Dropout(dropout))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class UNetUp(nn.Module):
    def __init__(self, in_size, out_size, dropout=0.0):
        super(UNetUp, self).__init__()
        layers = [
            nn.ConvTranspose2d(in_size, out_size, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(out_size),
            nn.ReLU(inplace=True),
        ]
        if dropout:
            layers.append(nn.Dropout(dropout))

        self.model = nn.Sequential(*layers)

    def forward(self, x, skip_input):
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)

        return x

class GeneratorUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(GeneratorUNet, self).__init__()

        self.down1 = UNetDown(in_channels, 64, normalize=False)
        self.down2 = UNetDown(64, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 512, dropout=0.5)
        self.down5 = UNetDown(512, 512, dropout=0.5)
        self.down6 = UNetDown(512, 512, dropout=0.5)
        self.down7 = UNetDown(512, 512, dropout=0.5)
        self.down8 = UNetDown(512, 512, normalize=False, dropout=0.5)

        self.up1 = UNetUp(512, 512, dropout=0.5)
        self.up2 = UNetUp(1024, 512, dropout=0.5)
        self.up3 = UNetUp(1024, 512, dropout=0.5)
        self.up4 = UNetUp(1024, 512, dropout=0.5)
        self.up5 = UNetUp(1024, 256)
        self.up6 = UNetUp(512, 128)
        self.up7 = UNetUp(256, 64)

        self.final = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(128, out_channels, 4, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)
        u1 = self.up1(d8, d7)
        u2 = self.up2(u1, d6)
        u3 = self.up3(u2, d5)
        u4 = self.up4(u3, d4)
        u5 = self.up5(u4, d3)
        u6 = self.up6(u5, d2)
        u7 = self.up7(u6, d1)
        return self.final(u7)

class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, normalization=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels * 2, 64, normalization=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1, bias=False)
        )

    def forward(self, img_A, img_B):
        img_input = torch.cat((img_A, img_B), 1)
        return self.model(img_input)

In [8]:
generator = GeneratorUNet().to(device)
discriminator = Discriminator().to(device)

In [9]:
def discriminator_train_step(real_src, real_trg, fake_trg):
    #discriminator.train()
    d_optimizer.zero_grad()

    prediction_real = discriminator(real_trg, real_src)
    error_real = criterion_GAN(prediction_real, torch.ones(len(real_src), 1, 16, 16).cuda())
    error_real.backward()

    prediction_fake = discriminator(fake_trg.detach(), real_src)
    error_fake = criterion_GAN(prediction_fake, torch.zeros(len(real_src), 1, 16, 16).cuda())
    error_fake.backward()

    d_optimizer.step()

    return error_real + error_fake

def generator_train_step(real_src, fake_trg):
    #discriminator.train()
    g_optimizer.zero_grad()
    prediction = discriminator(fake_trg, real_src)

    loss_GAN = criterion_GAN(prediction, torch.ones(len(real_src), 1, 16, 16).cuda())
    loss_pixel = criterion_pixelwise(fake_trg, real_trg)
    loss_G = loss_GAN + lambda_pixel * loss_pixel

    loss_G.backward()
    g_optimizer.step()
    return loss_G

denorm = T.Normalize((-1, -1, -1), (2, 2, 2))
def sample_prediction():
    """Saves a generated sample from the validation set"""
    data = next(iter(val_dl))
    real_src, real_trg = data
    fake_trg = generator(real_src)
    img_sample = torch.cat([denorm(real_src[0]), denorm(fake_trg[0]), denorm(real_trg[0])], -1)
    img_sample = img_sample.detach().cpu().permute(1,2,0).numpy()
    show(img_sample, title='Source::Generated::GroundTruth', sz=12)

In [10]:
generator = GeneratorUNet().to(device)
discriminator = Discriminator().to(device)
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

criterion_GAN = torch.nn.MSELoss()
criterion_pixelwise = torch.nn.L1Loss()

lambda_pixel = 100
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

val_dl = DataLoader(val_ds, batch_size=1, shuffle=True)

epochs = 100
log = Report(epochs)

for epoch in range(epochs):
    N = len(trn_dl)
    for bx, batch in enumerate(trn_dl):
        real_src, real_trg = batch
        fake_trg = generator(real_src)
        
        errD = discriminator_train_step(real_src, real_trg, fake_trg)
        errG = generator_train_step(real_src, fake_trg)
        log.record(pos=epoch+(1+bx)/N, errD=errD.item(), errG=errG.item(), end='\r')

    log.report_avgs(epoch+1)
    [sample_prediction() for _ in range(2)]

Output hidden; open in https://colab.research.google.com to view.